In [333]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
#from sklearn.linear_model import LogisticRegression

In [334]:
df = pd.read_csv('dataset/covid.csv')
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [335]:
#trf1 = df.fillna(method='ffill',inplace=True)

In [336]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [337]:
x_train,x_test,y_train,y_test=train_test_split(df.iloc[:,:5],df.iloc[:,-1],train_size=0.8,random_state=1)

In [338]:
x_train

,age,gender,fever,cough,city
2,42,Male,101.0,Mild,Delhi
73,34,Male,98.0,Strong,Kolkata
97,20,Female,101.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
19,42,Female,NaN,Strong,Bangalore
...,...,...,...,...,...
75,5,Male,102.0,Mild,Kolkata
9,64,Female,101.0,Mild,Delhi
72,83,Female,101.0,Mild,Kolkata
12,25,Female,99.0,Strong,Kolkata


In [339]:
trf1 = ColumnTransformer(transformers=[
    ('fever_si',SimpleImputer(),[2])
],remainder='passthrough')

In [340]:
trf2 = ColumnTransformer(transformers=[
        ('gender_city_ohe',OneHotEncoder(sparse=False,handle_unknown='ignore'),[2,4]),
        ('cough_oe',OrdinalEncoder(categories=[['Mild','Strong']]),[3])
],remainder='passthrough')

In [341]:
trf3 = ColumnTransformer(transformers=[
    ('age_scale',MinMaxScaler(),[0])
],remainder='passthrough')

In [342]:
trf4 = SelectKBest(score_func=chi2,k=1)

In [343]:
from sklearn.tree import DecisionTreeClassifier
trf5 = DecisionTreeClassifier()

In [344]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [349]:
from sklearn import set_config
set_config(display='diagram')

In [350]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('fever_si', SimpleImputer(),
                                                  [2])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('gender_city_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [2, 4]),
                                                 ('cough_oe',
                                                  OrdinalEncoder(categories=[['Mild',
                                                                              'Strong']]),
                                                  [3])])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('age_scale', MinMaxScaler(),
                                                  [0])])),
                ('trf4',
                 SelectKBest(k=1,
                             score_func=<function chi2 at 0x0000023CC30A61F0>)),
                ('trf5', DecisionTreeClassifier())])

In [347]:
y_pred = pipe.predict(x_test)
y_pred

array(['No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
       'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No'],
      dtype=object)

In [348]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5